#Reyes Ceballos            
#Anna Jadonath
#CUS 615 Nikhil Yadav

# Predicting the Default of Taiwanese Credit Card Holders

In [1]:
import pandas

credit = pandas.read_excel('default of credit card clients- feature selection .xlsx')


In [2]:
#only use for not normalized data
Xcolumns = credit.iloc[:,0:-1] #data
Ycolumn = credit.iloc[:,-1] #class

Xcolumns = (Xcolumns - Xcolumns.mean()) / Xcolumns.std()
# Normalize all of the numeric columns
# (obs - mean) / stand Dev

In [4]:
# Split data into training and test sets

import random 
from numpy.random import permutation
from sklearn.model_selection import train_test_split

#Xcolumns = credit.iloc[:,0:-1] #data
#Ycolumn = credit.iloc[:,-1] #class
Xtrain, Xtest, Ytrain,Ytest = train_test_split(Xcolumns,Ycolumn,test_size=0.1,random_state=52)

In [5]:
#performace evaluators
from sklearn.metrics import accuracy_score # (TP + TN) / total
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score # # % of Pred defaults(pos) that actually default TP / (TP + FP)


#from sklearn.metrics import recall_score # % of Pred defaults accurately predicted TP / (TP + FN) 
#specificity = % of Pred Paid accurately predicted TN / (TN + FP)

In [6]:
#KNN

from sklearn import neighbors

result = []

for i in [1,2,3,5,20,50,100,500,1500,3000]: # [500,1500,3000] accuracy goes down
    n_neighbors = i
    KNN = neighbors.KNeighborsClassifier(n_neighbors)

    KNN.fit(Xtrain,Ytrain) #use training set not entire dataset of X,y

    # test our training data
    Y_predictions = KNN.predict(Xtrain)
    train_acc1 = accuracy_score(Y_predictions, Ytrain)
    
    mse = (((Y_predictions - Ytrain) ** 2).sum()) / len(Y_predictions)
    
    print("Training Set Accuracy for KNN: ",i,'nearest neighbors',round(train_acc1,3),'with MSE of ',round(mse,3))
    
    #test our test set data vs class
    Y_predictions = KNN.predict(Xtest)
    test_acc1 = accuracy_score(Y_predictions, Ytest) 
    
    mse = (((Y_predictions - Ytest) ** 2).sum()) / len(Y_predictions)
    # Compute the mean squared error of our predictions.    
    
    print("    Test Set Accuracy for KNN: ",i,'nearest neighbors',round(test_acc1,3),'with MSE of ',round(mse,3))
    
    scores = cross_val_score(KNN,Xcolumns,Ycolumn,cv = 10)
    print('Cross Validation average ',round(scores.mean(),5))

    precision = precision_score(y_true = Ytest, y_pred = Y_predictions)    
    print('Precision ',precision,'\n') 
        
    result.append([i,test_acc1,mse,precision,scores.mean()])
    
result = pandas.DataFrame(result)
result.to_csv('KNNresults.csv')
    
  

Training Set Accuracy for KNN:  1 nearest neighbors 0.999 with MSE of  0.001
    Test Set Accuracy for KNN:  1 nearest neighbors 0.721 with MSE of  0.279
Cross Validation average  0.72317
Precision  0.369565217391 

Training Set Accuracy for KNN:  2 nearest neighbors 0.859 with MSE of  0.141
    Test Set Accuracy for KNN:  2 nearest neighbors 0.777 with MSE of  0.223
Cross Validation average  0.77927
Precision  0.502024291498 

Training Set Accuracy for KNN:  3 nearest neighbors 0.859 with MSE of  0.141
    Test Set Accuracy for KNN:  3 nearest neighbors 0.767 with MSE of  0.233
Cross Validation average  0.76837
Precision  0.469696969697 

Training Set Accuracy for KNN:  5 nearest neighbors 0.837 with MSE of  0.163
    Test Set Accuracy for KNN:  5 nearest neighbors 0.786 with MSE of  0.214
Cross Validation average  0.78497
Precision  0.540616246499 

Training Set Accuracy for KNN:  20 nearest neighbors 0.811 with MSE of  0.189
    Test Set Accuracy for KNN:  20 nearest neighbors 0.794

In [7]:
#Best KNN is 20 nearest neighbors as it has the highest test accuracy, low mse and a similar cross validation score

In [8]:
# SVM ***takes a while to run because of its size

from sklearn import svm

result = []

kernels = ['sigmoid','rbf'] # 'precomputed','linear','poly'

#'linear','poly' take to long to run
#'precomputed' ValueError: X.shape[0] should be equal to X.shape[1]

for i in kernels:
    SVM = svm.SVC(kernel=i) #model
    SVM.fit(Xtrain,Ytrain) #use training set not entire dataset of X,y

    #test our train set data vs class
    Y_Pred_Train = SVM.predict(Xtrain)
    train_acc2 = round(accuracy_score(Y_Pred_Train, Ytrain),4)
    mse = (((Y_Pred_Train - Ytrain) ** 2).sum()) / len(Y_Pred_Train)    
    print("Training Set Accuracy for SVM with kernel "+ i + ' '+str(round(train_acc2,3))+' with MSE of ',round(mse,3))

    #test our test set data vs class
    Y_Pred_Test = SVM.predict(Xtest) #to predict put data in here and print
    test_acc2 = accuracy_score(Y_Pred_Test, Ytest)
    mse = (((Y_Pred_Test - Ytest) ** 2).sum()) / len(Y_Pred_Test)    
    print("    Test Set Accuracy for SVM with kernel "+ i+' ' +str(round(test_acc2,3)),' with MSE of ',round(mse,3))
    
    scores = cross_val_score(SVM,Xcolumns,Ycolumn, cv=10)
    print('Cross Validation average ',round(scores.mean(),5))

    precision = precision_score(y_true = Ytest, y_pred = Y_Pred_Test)    
    print('Precision ',precision,'\n') 
        
    result.append([i,test_acc2,mse,precision,scores.mean()])

result = pandas.DataFrame(result)
result.to_csv('SVMresults.csv')

Training Set Accuracy for SVM with kernel sigmoid 0.703 with MSE of  0.297
    Test Set Accuracy for SVM with kernel sigmoid 0.708  with MSE of  0.292
Cross Validation average  0.7028
Precision  0.341538461538 

Training Set Accuracy for SVM with kernel rbf 0.805 with MSE of  0.195
    Test Set Accuracy for SVM with kernel rbf 0.799  with MSE of  0.201
Cross Validation average  0.8023
Precision  0.648888888889 



In [9]:
# Had to use kernel rbf and sigmoid as the other kernels took too long to run or did not run
#SVM with rbf slightly outperforms KNN


In [10]:
# Logistic Regression

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(Xtrain,Ytrain) #uses training data from these 2 columns


#test our train set data vs class
Y_Pred_Train = lr.predict(Xtrain)
train_accLR = round(accuracy_score(Y_Pred_Train, Ytrain),4)
mse = (((Y_Pred_Train - Ytrain) ** 2).sum()) / len(Y_Pred_Train)    
print("Training Set Accuracy for Logistic Regression " +str(round(train_accLR,3)),'with MSE of ',round(mse,3))


#test our test set data vs class
Y_Pred_Test = lr.predict(Xtest) #to predict put data in here and print
test_accLR = accuracy_score(Y_Pred_Test, Ytest)
mse = (((Y_Pred_Test - Ytest) ** 2).sum()) / len(Y_Pred_Test)    
print("Testing Accuracy for Logistic Regression " +str(round(test_accLR,3)),'with MSE of ',round(mse,3))

scores = cross_val_score(lr,Xcolumns,Ycolumn, cv=10)
print('Cross Validation average ',round(scores.mean(),5))

precision = precision_score(y_true = Ytest, y_pred = Y_Pred_Test)   
print('Precision ',precision,'\n') 

result = [test_accLR,mse,precision,scores.mean()]

result = pandas.DataFrame(result)              
result.to_csv('LRresults.csv')

'''import statsmodels.formula.api as sm

model = sm.logit(formula='Ycolumn ~ Xcolumns', data = credit)
fitted = model.fit()
fitted.summary()'''
#provides summary statistics for predicting 'Ycolumn' based on 'Xcolumns' using all data

Training Set Accuracy for Logistic Regression 0.799 with MSE of  0.201
Testing Accuracy for Logistic Regression 0.794 with MSE of  0.206
Cross Validation average  0.79807
Precision  0.666666666667 



"import statsmodels.formula.api as sm\n\nmodel = sm.logit(formula='Ycolumn ~ Xcolumns', data = credit)\nfitted = model.fit()\nfitted.summary()"

In [11]:
# Logistic regression is not good as SVM but performs better than KNN
#Not sure how to read summary statistics

In [12]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

result = []

for leaves in [3,10,50]:
    for i in [10,30,50]: # Number of trees
        print('Number of leaves ',leaves)
        rf = RandomForestClassifier(n_estimators=i,min_samples_leaf=leaves)
        #n_estimators number of random forest

        #training
        rf = rf.fit(Xtrain,Ytrain)
        predictions = rf.predict(Xtrain)
        train_acc3 = round(accuracy_score(predictions, Ytrain),4)
        mse = (((predictions - Ytrain) ** 2).sum()) / len(predictions)
        print("Training Accuracy for Random Forest with trees of ",i,' ',train_acc3,'with MSE of ',round(mse,3))

        predictions = rf.predict(Xtest)
        test_acc3 = round(accuracy_score(predictions, Ytest),4)
        mse = (((predictions - Ytest) ** 2).sum()) / len(predictions)
        print(" Testing Accuracy for Random Forest with trees of ",i,' ',test_acc3,'with MSE of ',round(mse,3))

        scores = cross_val_score(rf,Xcolumns,Ycolumn, cv=10)
        print('Cross Validation average ',round(scores.mean(),5))

        precision = precision_score(y_true = Ytest, y_pred = predictions)    
        print('Precision ',precision,'\n') 

        result.append([leaves,i,test_acc3,mse,precision,scores.mean()])
    
result = pandas.DataFrame(result)
result.to_csv('RFresults.csv')

Number of leaves  3
Training Accuracy for Random Forest with trees of  10   0.8909 with MSE of  0.109
 Testing Accuracy for Random Forest with trees of  10   0.8033 with MSE of  0.197
Cross Validation average  0.79764
Precision  0.612994350282 

Number of leaves  3
Training Accuracy for Random Forest with trees of  30   0.8929 with MSE of  0.107
 Testing Accuracy for Random Forest with trees of  30   0.7977 with MSE of  0.202
Cross Validation average  0.80323
Precision  0.594029850746 

Number of leaves  3
Training Accuracy for Random Forest with trees of  50   0.8919 with MSE of  0.108
 Testing Accuracy for Random Forest with trees of  50   0.7987 with MSE of  0.201
Cross Validation average  0.8052
Precision  0.598214285714 

Number of leaves  10
Training Accuracy for Random Forest with trees of  10   0.8362 with MSE of  0.164
 Testing Accuracy for Random Forest with trees of  10   0.7923 with MSE of  0.208
Cross Validation average  0.8039
Precision  0.574603174603 

Number of leaves 

In [13]:
#despite having the highest training accuracy (90% at 3 leaves), it's testing set accuracy/mse is similar to SVM
#50 leaves is the least overfit

In [14]:
#Kmeans

from sklearn.cluster import KMeans    

result = []
    
for i in [1,2,3,4,5,10]:# number of clusters

    Kmeans = KMeans(n_clusters=i, random_state=52)
    Kmeans.fit(Xtrain) 

    #test our train set data vs class
    Y_Pred_Train = Kmeans.predict(Xtrain)
    train_acc4 = round(accuracy_score(Y_Pred_Train, Ytrain),4)
    mse = (((Y_Pred_Train - Ytrain) ** 2).sum()) / len(Y_Pred_Train)    
    print("Train Set Accuracy for Kmeans with K of ",i,': '+str(train_acc4),'with MSE of ',round(mse,3))

    #test our test set data vs class
    Y_Pred_Test = Kmeans.predict(Xtest) #to predict put data in here and print
    test_acc4 = accuracy_score(Y_Pred_Test, Ytest)
    mse = (((Y_Pred_Test - Ytest) ** 2).sum()) / len(Y_Pred_Test)    
    print(" Test Set Accuracy for Kmeans with K of ",i,': '+str(round(test_acc4,3)),'with MSE of ',round(mse,3))

    #print('precision ',precision_score(y_true = Ytest, y_pred = Y_Pred_Test))
    
    scores = cross_val_score(Kmeans,Xcolumns,Ycolumn, cv=3)
    print('Cross Validation average ',round(scores.mean(),5))

    if i < 3:
        precision = precision_score(y_true = Ytest, y_pred = Y_Pred_Test)
    else:
        precision = precision_score(y_true = Ytest, y_pred = Y_Pred_Test,average = 'macro')
        
    print('Precision ',precision,'\n') 

    result.append([i,test_acc4,mse,precision,scores.mean()])
    
result = pandas.DataFrame(result)
result.to_csv('KMeansresults.csv')

Train Set Accuracy for Kmeans with K of  1 : 0.779 with MSE of  0.221
 Test Set Accuracy for Kmeans with K of  1 : 0.777 with MSE of  0.223
Cross Validation average  -80150.20674
Precision  0.0 



C:\Users\RCeballos\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Train Set Accuracy for Kmeans with K of  2 : 0.4383 with MSE of  0.562
 Test Set Accuracy for Kmeans with K of  2 : 0.448 with MSE of  0.552
Cross Validation average  -65318.13247
Precision  0.165193745751 

Train Set Accuracy for Kmeans with K of  3 : 0.2599 with MSE of  1.46
 Test Set Accuracy for Kmeans with K of  3 : 0.273 with MSE of  1.438
Cross Validation average  -58189.12215
Precision  0.293603117558 

Train Set Accuracy for Kmeans with K of  4 : 0.266 with MSE of  3.122
 Test Set Accuracy for Kmeans with K of  4 : 0.265 with MSE of  3.07
Cross Validation average  -53133.93408
Precision  0.259667691889 

Train Set Accuracy for Kmeans with K of  5 : 0.2816 with MSE of  3.583
 Test Set Accuracy for Kmeans with K of  5 : 0.284 with MSE of  3.502
Cross Validation average  -49054.90271
Precision  0.230567651195 

Train Set Accuracy for Kmeans with K of  10 : 0.0696 with MSE of  29.196
 Test Set Accuracy for Kmeans with K of  10 : 0.064 with MSE of  29.774
Cross Validation average  

In [15]:
#Kmeans works best with K of 1 it has an accuracy and mse similar to KNN
#cross validation score is strange
#Not the best model for this

In [16]:
#Neural Nets
#http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier

from sklearn.neural_network import MLPClassifier

result = []

activation = ['identity','logistic','tanh','relu']
solver = ['lbfgs','sgd','adam']

for i in activation:
    for j in solver:
    
        NeuralNet = MLPClassifier(activation = i,solver = j, random_state=1, hidden_layer_sizes = 10, max_iter=1000)
        NeuralNet.fit(Xtrain,Ytrain) 
        
        print('For Neural Network of activiation ',i,' and solver of ',j)
        
        #test our train set data vs class
        Y_Pred_Train = NeuralNet.predict(Xtrain)
        train_acc5 = round(accuracy_score(Y_Pred_Train, Ytrain),4)
        mse = (((Y_Pred_Train - Ytrain) ** 2).sum()) / len(Y_Pred_Train)    
        print("Train Set Accuracy for Neural Network "+str(train_acc5),'with MSE of ',round(mse,3))

        #test our test set data vs class
        Y_Pred_Test = NeuralNet.predict(Xtest) #to predict put data in here and print
        test_acc5 = accuracy_score(Y_Pred_Test, Ytest)
        mse = (((Y_Pred_Test - Ytest) ** 2).sum()) / len(Y_Pred_Test)    
        print(" Test Set Accuracy for Neural Network "+str(round(test_acc5,3)),'with MSE of ',round(mse,3))

        scores = cross_val_score(NeuralNet,Xcolumns,Ycolumn, cv=10)
        precision = precision_score(y_true = Ytest, y_pred = Y_Pred_Test)    
        print('Precision ',precision,'\n') 

        result.append([i,j,test_acc5,mse,precision,scores.mean()])
    
result = pandas.DataFrame(result)
result.to_csv('NeuralNetsresults.csv')
        

For Neural Network of activiation  identity  and solver of  lbfgs
Train Set Accuracy for Neural Network 0.7988 with MSE of  0.201
 Test Set Accuracy for Neural Network 0.794 with MSE of  0.206
Precision  0.666666666667 

For Neural Network of activiation  identity  and solver of  sgd
Train Set Accuracy for Neural Network 0.7987 with MSE of  0.201
 Test Set Accuracy for Neural Network 0.794 with MSE of  0.206
Precision  0.664516129032 

For Neural Network of activiation  identity  and solver of  adam
Train Set Accuracy for Neural Network 0.7995 with MSE of  0.201
 Test Set Accuracy for Neural Network 0.795 with MSE of  0.205
Precision  0.66091954023 

For Neural Network of activiation  logistic  and solver of  lbfgs
Train Set Accuracy for Neural Network 0.8077 with MSE of  0.192
 Test Set Accuracy for Neural Network 0.801 with MSE of  0.199
Precision  0.603448275862 

For Neural Network of activiation  logistic  and solver of  sgd
Train Set Accuracy for Neural Network 0.7977 with MSE of

In [17]:
# Neural Network is pretty good at all activations and solvers

In [18]:
# Gaussian Naive Bayes

from sklearn.naive_bayes import GaussianNB

result = []
Bayes = GaussianNB()
Bayes.fit(Xtrain,Ytrain)

#test our train set data vs class
Y_Pred_Train = Bayes.predict(Xtrain)
train_acc5 = round(accuracy_score(Y_Pred_Train, Ytrain),4)
mse = (((Y_Pred_Train - Ytrain) ** 2).sum()) / len(Y_Pred_Train)    
print("Train Set Accuracy for Gaussian Naive Bayes "+str(round(train_acc5,3)),'with MSE of ',round(mse,3))

#test our test set data vs class
Y_Pred_Test = Bayes.predict(Xtest) #to predict put data in here and print
test_acc5 = accuracy_score(Y_Pred_Test, Ytest)
mse = (((Y_Pred_Test - Ytest) ** 2).sum()) / len(Y_Pred_Test)    
print(" Test Set Accuracy for Gaussian Naive Bayes "+str(round(test_acc5,3)),'with MSE of ',round(mse,3))

scores = cross_val_score(Bayes,Xcolumns,Ycolumn, cv=10)
print('Cross Validation average ',round(scores.mean(),5))

precision = precision_score(y_true = Ytest, y_pred = Y_Pred_Test)
print('Precision ',precision,'\n') 

result.append([test_acc5,mse,precision,scores.mean()])
    
result = pandas.DataFrame(result)
result.to_csv('NBresults.csv')


Train Set Accuracy for Gaussian Naive Bayes 0.802 with MSE of  0.198
 Test Set Accuracy for Gaussian Naive Bayes 0.802 with MSE of  0.198
Cross Validation average  0.80117
Precision  0.616564417178 



Lab 1 - 
    Correlation
    Kmeans with PCA
    linear regression
    Random forest
    MSE mean squared error
    
Lab 2 - 
    Hist
    Boxplot
    Desc stats
    
Lab 3 - 
    KNN
    normalization
    
Lab 4 - 
    SVM
    scattplot
    performance
    KNN
    
Lab 5 - Neural Nets 

filter class and see if there are patterns?
